In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import sympy as sp

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

In [3]:
t = sp.Symbol('t', real=True)

q1 = sp.Function('q1')(t)
q2 = sp.Function('q2')(t)
q3 = sp.Function('q3')(t)

dq1 = sp.diff(q1, t)
dq2 = sp.diff(q2, t)
dq3 = sp.diff(q3, t)

m1, m2, m3 = sp.symbols('m1 m2 m3', positive=True)
l1, l2 = sp.symbols('l1 l2', positive=True)
a1, a2, a3 = sp.symbols('a1 a2 a3', real=True)
g = sp.symbols('g', positive=True)

I1 = sp.symbols('I1', positive=True)
I2 = sp.symbols('I2', positive=True)
I3 = sp.symbols('I3', positive=True)

In [5]:
x1 = a1 * sp.cos(q1)
x2 = l1 * sp.cos(q1) + a2 * sp.cos(q1 + q2)
x3 = l1 * sp.cos(q1) + l2 * sp.cos(q1 + q2) + a3 * sp.cos(q1 + q2 - q3)

y1 = a1 * sp.sin(q1)
y2 = l1 * sp.sin(q1) + a2 * sp.sin(q1 + q2)
y3 = l1 * sp.sin(q1) + l2 * sp.sin(q1 + q2) + a3 * sp.sin(q1 + q2 - q3)

dx1 = sp.diff(x1, t)
dx2 = sp.diff(x2, t)
dx3 = sp.diff(x3, t)

dy1 = sp.diff(y1, t)
dy2 = sp.diff(y2, t)
dy3 = sp.diff(y3, t)

dx1 = sp.simplify(dx1)
dy1 = sp.simplify(dy1)
dx2 = sp.simplify(dx2)
dy2 = sp.simplify(dy2)
dx3 = sp.simplify(dx3)
dy3 = sp.simplify(dy3)


In [6]:
v1_2 = sp.simplify(dx1**2 + dy1**2)
v2_2 = sp.simplify(dx2**2 + dy2**2)
v3_2 = sp.simplify(dx3**2 + dy3**2)

In [9]:
K1 = 0.5 * (m1 * v1_2 + I1 * dq1**2)
K2 = 0.5 * (m2 * v2_2 + I2 * (dq1**2 + dq2**2))
K3 = 0.5 * (m3 * v3_2 + I3 * (dq1**2 + dq2**2 - dq3**2))
K = sp.simplify(K1 + K2 + K3)

P1 = m1 * g * y1
P2 = m2 * g * y2
P3 = m3 * g * y3
P = sp.simplify(P1 + P2 + P3)

L = K - P

equation1 = sp.diff(sp.diff(L, dq1), t) - sp.diff(L, q1)
equation2 = sp.diff(sp.diff(L, dq2), t) - sp.diff(L, q2)
equation3 = sp.diff(sp.diff(L, dq3), t) - sp.diff(L, q3)

equation1 = sp.simplify(equation1)
equation2 = sp.simplify(equation2)
equation3 = sp.simplify(equation3)

equation1 = sp.simplify(equation1)
equation2 = sp.simplify(equation2)
equation3 = sp.simplify(equation3)

In [15]:
print(equation3)


-1.0*I3*Derivative(q3(t), (t, 2)) - 1.0*a3**2*m3*Derivative(q1(t), (t, 2)) - 1.0*a3**2*m3*Derivative(q2(t), (t, 2)) + 1.0*a3**2*m3*Derivative(q3(t), (t, 2)) - 1.0*a3*g*m3*cos(q1(t) + q2(t) - q3(t)) - 1.0*a3*l1*m3*sin(q2(t) - q3(t))*Derivative(q1(t), t)**2 - 1.0*a3*l1*m3*cos(q2(t) - q3(t))*Derivative(q1(t), (t, 2)) + 1.0*a3*l2*m3*sin(q3(t))*Derivative(q1(t), t)**2 + 2.0*a3*l2*m3*sin(q3(t))*Derivative(q1(t), t)*Derivative(q2(t), t) + 1.0*a3*l2*m3*sin(q3(t))*Derivative(q2(t), t)**2 - 1.0*a3*l2*m3*cos(q3(t))*Derivative(q1(t), (t, 2)) - 1.0*a3*l2*m3*cos(q3(t))*Derivative(q2(t), (t, 2))


In [16]:
ddq = [sp.diff(q1, t, 2), sp.diff(q2, t, 2), sp.diff(q3, t, 2)]
dq = [dq1, dq2, dq3]
equations = [equation1, equation2, equation3]

In [17]:
M = sp.zeros(3, 3)
C_terms = [sp.zeros(3, 1) for _ in range(3)]  # Временное хранение для C
G = sp.zeros(3, 1)

for i, eq in enumerate(equations):
    # 1. Извлекаем M (коэффициенты при q̈)
    for j in range(3):
        # Собираем все члены с ddq[j]
        collected = sp.collect(eq.expand(), ddq[j], evaluate=False)
        M[i, j] = collected.get(ddq[j], 0)
    
    # 2. Удаляем члены с q̈
    eq_no_ddq = eq - sum(M[i, j] * ddq[j] for j in range(3))
    
    # 3. Извлекаем G (члены без q̇)
    G[i] = eq_no_ddq.subs([(dq1, 0), (dq2, 0), (dq3, 0)])
    
    # 4. Остаток - это C(q, q̇)q̇
    residual = eq_no_ddq - G[i]
    
    # 5. Для построения C нужно разложить residual
    # residual = C[i, 0]*dq1 + C[i, 1]*dq2 + C[i, 2]*dq3
    # Но могут быть нелинейные члены типа dq1^2, dq1*dq2
    
    # Собираем по каждой производной
    for j in range(3):
        # Находим все члены, содержащие dq[j]
        terms = sp.collect(residual, dq[j], evaluate=False)
        if dq[j] in terms:
            coeff = terms[dq[j]]
            # Если coeff содержит другие dq, это нелинейный член
            # В этом случае C[i, j] будет зависеть от q̇
            C_terms[i][j] = coeff
        else:
            C_terms[i][j] = 0

# Формируем матрицу C
# В общем случае C[i, j] может зависеть от q̇, поэтому это матрица функций
C = sp.Matrix([[C_terms[i][j] for j in range(3)] for i in range(3)])

# Упрощаем
M = sp.simplify(M)
C = sp.simplify(C)
G = sp.simplify(G)


In [18]:
print("M(q):")
sp.pprint(M)
print("\nC(q, q_dot):")
sp.pprint(C)
print("\nG(q):")
sp.pprint(G)

M(q):
⎡                                 2            2                               ↪
⎢1.0⋅I₁ + 1.0⋅I₂ + 1.0⋅I₃ + 1.0⋅a₁ ⋅m₁ + 1.0⋅a₂ ⋅m₂ + 2.0⋅a₂⋅l₁⋅m₂⋅cos(q₂(t))  ↪
⎢                                                                              ↪
⎢                                       2                                      ↪
⎢                                 1.0⋅a₂ ⋅m₂ + 1.0⋅a₂⋅l₁⋅m₂⋅cos(q₂(t)) + 1.0⋅a ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎣                                                                              ↪

↪         2                                                                    ↪
↪ + 1.0⋅a₃ ⋅m₃ + 2.0⋅a₃⋅l₁⋅m₃⋅cos(q₂(t) - q₃(t)) + 2.0⋅a₃⋅l₂⋅m₃⋅cos(q₃(t)) + 1 ↪
↪                                                                              ↪
↪  2                                                                           ↪
↪ ₃ ⋅m₃ + 1.0⋅a₃⋅l₁⋅m